### Preprocess

In [3]:
#4720 + 2606 = 7326
# import model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from PIL import Image
import random
#from sklearn.metrics import f1_score
#from matplotlib.animation import FuncAnimation
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
from sklearn.metrics import f1_score

## Rename all file 

In [14]:
def rename_and_number(folder_path, new_name):
    for count, filename in enumerate(os.listdir(folder_path)):
        old_file = os.path.join(folder_path, filename)
        # Ensure it's a file
        if os.path.isfile(old_file):
            # Get file extension
            extension = os.path.splitext(filename)[1]
            if extension != ".png":
                extension = ".png"
            # Create new file name
            new_file = os.path.join(folder_path, f"{new_name}_{count + 1}{extension}")
            # Rename the file
            os.rename(old_file, new_file)
    print("Files renamed successfully!")

In [5]:
rename_and_number("H:\我的雲端硬碟\jay chou\\total_Jay\\", "RealJay")
rename_and_number("H:\我的雲端硬碟\jay chou\\total_notJay", "FakeJay")

Read Data

In [13]:
def _preprocess(image_arr, th=0.4):
        image_arr[image_arr > th] = 1.0
        image_arr[image_arr <= th] = 0.0
        return image_arr

In [14]:
def SplitData(data):
    split_ratio = 0.1
    split_index = int((1 - split_ratio) * len(X_train))
    # Split the data into training and validation sets

    x_train = X_train[:split_index]
    y_train = Y_train[:split_index]
    x_val = X_train[split_index:]
    y_val = Y_train[split_index:]


In [15]:
from PIL import Image

def image_resize(path):
    # path : the image path you want to resize
    # img_resized: img have resize to 360*360
    #open picture
    img = Image.open(path)
    # resize to 360*360
    img_resized = img.resize((180, 180), Image.LANCZOS)
    return img_resized


In [16]:
def read_images_from_folder(folder_path):
    # read images from 'folder_path'
    # flat the image and push into 'images' array
    print(f'reading from {folder_path}...')
    images = []
    length = len(os.listdir(folder_path))
    i = 0
    for filename in os.listdir(folder_path):
        i = i + 1
        progress = ((i + 1)/length)*100
        if i % 500 == 0:
            print(f"progress: {progress} %")
        if filename.lower().endswith('.png'):
            file_path = os.path.join(folder_path, filename)
            try:
                # 打開圖片
                with Image.open(file_path) as img:
                    # 將圖片轉為灰階
                    img = img.convert('L')
                    # 檢查圖片尺寸是否為 360x360
                    if img.size != (180, 180):
                        img = image_resize(file_path)
                        img.save(file_path)
                        print(file_path)
                        print(img.size)
                    if img.size == (180, 180):
                        # 將圖片轉換為 np.array 並進行標準化 (值範圍從 0 到 1)
                        img_array = np.array(img) / 255.0
                        # 將圖片展平 (flatten)
                        img_array = img_array.flatten()
                        #print(img_array)
                        images.append(img_array)
                        #print(images[0])
                        #print(f'image size: {len(images[0])}')
                    else:
                        print(f"圖片尺寸為 {img.size}, 不是 180x180: {filename}")
            except UnidentifiedImageError:
                print(f"無法識別圖片文件: {filename}")
            except Exception as e:
                print(f"無法讀取圖片: {filename}, 錯誤: {e}")
    print(f"finish reading from {folder_path}")
    return images

In [26]:
folder_path_test = "G:\我的雲端硬碟\ML test\\"
preprocess_folder_real = "H:\我的雲端硬碟\jay chou\\total_Jay\\"
preprocess_folder_fake = "H:\我的雲端硬碟\jay chou\\total_notJay\\"
preprocess_fake = np.array(read_images_from_folder(preprocess_folder_fake), dtype="object")
print(preprocess_fake.shape)
preprocess_real = np.array(read_images_from_folder(preprocess_folder_real), dtype="object")

real_labels = np.ones((preprocess_real.shape[0], 1))  
fake_labels = np.ones((preprocess_fake.shape[0], 1))
fake_labels = fake_labels * (-1)
print(fake_labels)
preprocess_photo = np.concatenate((preprocess_fake, preprocess_real), axis=0)
print(preprocess_photo.shape)

labels = np.concatenate((fake_labels, real_labels), axis=0)
print(labels.shape)
for i in range(len(preprocess_photo)):
    preprocess_photo[i] = _preprocess(preprocess_photo[i])
#preprocess_photo


reading from H:\我的雲端硬碟\jay chou\total_notJay\...
progress: 19.217491369390103 %
progress: 38.39662447257383 %
progress: 57.57575757575758 %
progress: 76.75489067894131 %
progress: 95.93402378212504 %
finish reading from H:\我的雲端硬碟\jay chou\total_notJay\
(2606, 32400)
reading from H:\我的雲端硬碟\jay chou\total_Jay\...
progress: 10.612158441008262 %
progress: 21.203134929040456 %
progress: 31.794111417072656 %
progress: 42.385087905104854 %
progress: 52.976064393137044 %
progress: 63.56704088116925 %
progress: 74.15801736920145 %
progress: 84.74899385723363 %
progress: 95.33997034526584 %
finish reading from H:\我的雲端硬碟\jay chou\total_Jay\
[[-1.]
 [-1.]
 [-1.]
 ...
 [-1.]
 [-1.]
 [-1.]]
(7326, 32400)
(7326, 1)


In [27]:

preprocess_photo = np.concatenate((preprocess_fake, preprocess_real), axis=0)
print(preprocess_photo.shape)

labels = np.concatenate((fake_labels, real_labels), axis=0)
print(labels.shape)
#for i in range(len(preprocess_photo)):
#    preprocess_photo[i] = _preprocess(preprocess_photo[i])
#preprocess_photo
print(preprocess_photo)

(7326, 32400)
(7326, 1)
[[0.2235294117647059 0.2235294117647059 0.2235294117647059 ...
  0.34901960784313724 0.3686274509803922 0.3686274509803922]
 [0.0 0.0 0.0 ... 0.3176470588235294 0.32941176470588235
  0.32941176470588235]
 [1.0 1.0 1.0 ... 0.34901960784313724 0.3058823529411765
  0.403921568627451]
 ...
 [0.25098039215686274 0.2235294117647059 0.23529411764705882 ...
  0.396078431372549 0.43137254901960786 0.39215686274509803]
 [0.03137254901960784 0.03529411764705882 0.03137254901960784 ...
  0.25098039215686274 0.25098039215686274 0.25098039215686274]
 [0.43137254901960786 0.4 0.39215686274509803 ... 0.792156862745098
  0.788235294117647 0.8]]


In [ ]:

split_ratio = 0.1
split_index = int((1 - split_ratio) * len(preprocess_photo))
# Split the data into training and validation sets

permutation = list(np.random.permutation(preprocess_photo.shape[0]))
shuffled_X = preprocess_photo[permutation, : ]
shuffled_Y = labels[permutation, : ]

x_train = shuffled_X[:split_index]
y_train = shuffled_Y[:split_index]
x_val = shuffled_X[split_index:]
y_val = shuffled_Y[split_index:]

In [22]:
print(x_train.shape)
print(y_train.shape)

(6593, 32400)
(6593, 1)


### Model

In [ ]:
# neural network
# CNN
# SVM


In [1]:
class Dense():
    def __init__(self, n_x, n_y, seed=1):
        self.n_x = n_x
        self.n_y = n_y
        self.seed = seed
        self.initialize_parameters()

    def initialize_parameters(self):

        sd = np.sqrt(6.0 / (self.n_x + self.n_y))
        np.random.seed(self.seed)
        W = np.random.uniform(-sd, sd, (self.n_y, self.n_x)).T      # the transpose here is just for the code to be compatible with the old codes
        b = np.zeros((1, self.n_y))

        assert(W.shape == (self.n_x, self.n_y))
        assert(b.shape == (1, self.n_y))

        self.parameters = {"W": W, "b": b}

    def forward(self, A):
        # GRADED FUNCTION: linear_forward
        ### START CODE HERE ###
        W = self.parameters["W"]
        b = self.parameters["b"]
        Z = np.dot(A, W) + b # matrix multiplication and addition
        self.cache = (A, W, b)

        ### END CODE HERE ###

        assert(Z.shape == (A.shape[0], self.parameters["W"].shape[1]))

        return Z

    def backward(self, dZ):

        A_prev, W, b = self.cache
        m = A_prev.shape[0]

        # GRADED FUNCTION: linear_backward
        ### START CODE HERE ###
        self.dW = (1 / m) * np.dot(A_prev.T, dZ) #gradient of loss with respect to weights
        self.db = (1 / m) * np.sum(dZ, axis=0, keepdims=True) #gradient of loss with respect to bias
        dA_prev = np.dot(dZ, W.T)
        ### END CODE HERE ###

        assert (dA_prev.shape == A_prev.shape)
        assert (self.dW.shape == self.parameters["W"].shape)
        assert (self.db.shape == self.parameters["b"].shape)

        return dA_prev

    def update(self, learning_rate):
    # GRADED FUNCTION: linear_update_parameters
    ### START CODE HERE ###
      self.parameters["W"] = self.parameters["W"] - learning_rate * self.dW
      self.parameters["b"] = self.parameters["b"] - learning_rate * self.db
      ### END CODE HERE ###
class Activation():
    def __init__(self, activation_function, loss_function):
        self.activation_function = activation_function
        self.loss_function = loss_function
        self.cache = None

    def forward(self, Z):
        if self.activation_function == "sigmoid":
            # GRADED FUNCTION: sigmoid_forward
            ### START CODE HERE ###
            Z = np.array(Z, dtype=float)

            A = 1 / (1 + np.exp(-Z))
            self.cache = Z
            ### END CODE HERE ###

            return A
        elif self.activation_function == "relu":
            # GRADED FUNCTION: relu_forward
            ### START CODE HERE ###
            A = np.maximum(0, Z)
            self.cache = Z
            ### END CODE HERE ###

            assert(A.shape == Z.shape)

            return A
        elif self.activation_function == "softmax":
            # GRADED FUNCTION: softmax_forward
            ### START CODE HERE ###
            A = np.exp(Z - np.max(Z, axis=1, keepdims=True)) / np.sum(np.exp(Z - np.max(Z, axis=1, keepdims=True)), axis=1, keepdims=True)
            self.cache = Z
            ### END CODE HERE ###

            return A
        elif self.activation_function == "linear":
            self.cache = Z.copy()
            return Z

        else:
            raise ValueError(f"Unsupported activation function: {self.activation_function}")


    def backward(self, dA=None, Y=None):
        if self.activation_function == "sigmoid":
            # GRADED FUNCTION: sigmoid_backward
            ### START CODE HERE ###
            Z = np.array(self.cache)
            A = 1 / (1 + np.exp(-Z))
            dZ = dA * A * (1 - A)
            ### END CODE HERE ###

            assert (dZ.shape == Z.shape)

            return dZ

        elif self.activation_function == "relu":
            # GRADED FUNCTION: relu_backward
            ### START CODE HERE ###
            Z = self.cache
            dZ = dA * (Z > 0)
            ### END CODE HERE ###

            assert (dZ.shape == Z.shape)

            return dZ

        elif self.activation_function == "softmax":
            # GRADED FUNCTION: softmax_backward
            ### START CODE HERE ###
            Z = self.cache
            s = np.exp(Z - np.max(Z, axis=1, keepdims=True)) / np.sum(np.exp(Z - np.max(Z, axis=1, keepdims=True)), axis=1, keepdims=True)
            dZ = s - Y
            ### END CODE HERE ###

            assert (dZ.shape == self.cache.shape)

            return dZ

        elif self.activation_function == "linear":
            return dA

        else:
            raise ValueError(f"Unsupported activation function: {self.activation_function}")

class Model():
    def __init__(self, units, activation_functions, loss_function):
        self.units = units
        self.activation_functions = activation_functions
        self.loss_function = loss_function
        self.initialize_parameters()

    def initialize_parameters(self):
        self.linear = []        # Store all Dense layers (weights & biases)
        self.activation = []    # Store all activation function layers

        for i in range(len(self.units)-1):
            dense = Dense(self.units[i], self.units[i+1], i)
            self.linear.append(dense)

        for i in range(len(self.activation_functions)):
            self.activation.append(Activation(self.activation_functions[i], self.loss_function))

    def forward(self, X):
        A = X

        # GRADED FUNCTION: model_forward
        ### START CODE HERE ###
        for i in range(len(self.linear)):
            A = self.linear[i].forward(A)
            A = self.activation[i].forward(A)
        ### END CODE HERE ###

        return A

    def backward(self, AL=None, Y=None):
        L = len(self.linear)
        C = Y.shape[1]

        # assertions
        warning = 'Warning: only the following 3 combinations are allowed! \n \
                    1. binary classification: sigmoid + cross_entropy \n \
                    2. multi-class classification: softmax + cross_entropy \n \
                    3. regression: linear + mse'
        assert self.loss_function in ["cross_entropy", "mse"], "you're using undefined loss function!"
        if self.loss_function == "cross_entropy":
            if Y.shape[1] == 1:  # binary classification
                assert self.activation_functions[-1] == 'sigmoid', warning
            else:  # multi-class classification
                assert self.activation_functions[-1] == 'softmax', warning
                assert self.units[-1] == Y.shape[1], f"you should set last dim to {Y.shape[1]}(the number of classes) in multi-class classification!"
        elif self.loss_function == "mse":
            assert self.activation_functions[-1] == 'linear', warning
            assert self.units[-1] == Y.shape[1], "output dimension mismatch for regression!"

        # GRADED FUNCTION: model_backward
        ### START CODE HERE ###
        if self.activation_functions[-1] == "linear":
            # Initializing the backpropagation
            dAL = AL - Y
            # Lth layer (LINEAR) gradients. Inputs: "dAL". Outputs: "dA_prev"
            dA_prev = self.linear[-1].backward(dZ=dAL)

        elif self.activation_functions[-1] == "sigmoid":
            # Initializing the backpropagation
            dAL = -(Y / (AL + 1e-5) - (1 - Y) / (1 - AL + 1e-5))

            # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL". Outputs: "dA_prev"
            dZ = self.activation[-1].backward(dA=dAL)
            dA_prev = self.linear[-1].backward(dZ=dZ)

        elif self.activation_functions[-1] == "softmax":
            # Initializing the backpropagation
            dZ = self.activation[-1].backward(Y=Y)

            # Lth layer (LINEAR) gradients. Inputs: "dZ". Outputs: "dA_prev"
            dA_prev = self.linear[-1].backward(dZ=dZ)

        # Loop from l=L-2 to l=0
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "dA_prev". Outputs: "dA_prev"
        for i in (range(L-2, -1, -1)):
            dZ = self.activation[i].backward(dA=dA_prev)
            dA_prev = self.linear[i].backward(dZ=dZ)
        ### END CODE HERE ###

        return dA_prev

    def update(self, learning_rate):
        L = len(self.linear)

        # GRADED FUNCTION: model_update_parameters
        ### START CODE HERE ###
        for i in range(L):
            self.linear[i].update(learning_rate)
        ### END CODE HERE ###
# GRADED FUNCTION: compute_BCE_loss

def compute_BCE_loss(AL, Y):
    n = Y.shape[0]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 line of code)
    loss = -(1/n) * np.sum(np.multiply(Y, np.log(AL + 1e-5)) + np.multiply(1 - Y, np.log(1 - AL + 1e-5)))
    ### END CODE HERE ###

    loss = np.squeeze(loss)      # To make sure your loss's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(loss.shape == ())

    return loss
# GRADED FUNCTION: compute_CCE_loss

def compute_CCE_loss(AL, Y):
    n = Y.shape[0]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 line of code)
    loss = -(1/n) * np.sum(np.multiply(Y, np.log(AL + 1e-5)))
    ### END CODE HERE ###

    loss = np.squeeze(loss)      # To make sure your loss's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(loss.shape == ())

    return loss
    
# compute_MSE_loss (MSE)
def compute_MSE_loss(AL, Y):
    m = Y.shape[0]
    loss = (1/m) * np.sum(np.square(AL - Y))
    return loss

def random_mini_batches(X, Y, mini_batch_size = 64):
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    ### START CODE HERE ###

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]
    shuffled_Y = Y[permutation, :]

    # Step 2 - Partition (shuffled_X, shuffled_Y).
    # Cases with a complete mini batch size only i.e each of 64 examples.
    num_complete_minibatches = math.floor(m / mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : (k+1) * mini_batch_size, :]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : (k+1) * mini_batch_size, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # For handling the end case (last mini-batch < mini_batch_size i.e less than 64)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m, :]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    ### END CODE HERE ###

    return mini_batches

def train_model(model, X_train, Y_train, learning_rate, num_iterations, batch_size=None, print_loss=True, print_freq=1000, decrease_freq=100, decrease_proportion=0.99):
    history = []
    losses = []

    for i in range(num_iterations):
        ### START CODE HERE ###
        # Define mini batches
        if batch_size:
            mini_batches = random_mini_batches(X_train, Y_train, batch_size)
        else:
            # if batch_size is None, batch is not used, mini_batch = whole dataset
            mini_batches = [(X_train, Y_train)]

        epoch_loss = 0
        for batch in mini_batches:
            X_batch, Y_batch = batch

            # Forward pass
            AL = model.forward(X_batch)

            # Compute loss
            if model.loss_function == 'cross_entropy':
                if model.activation_functions[-1] == "sigmoid": # Binary classification
                    loss = compute_BCE_loss(AL, Y_batch)
                elif model.activation_functions[-1] == "softmax": # Multi-class classification
                    loss = compute_CCE_loss(AL, Y_batch)
            elif model.loss_function == 'mse': # Regression
                loss = compute_MSE_loss(AL, Y_batch)
            epoch_loss += loss

            # Backward pass
            model.backward(AL, Y_batch)

            # Update parameters
            model.update(learning_rate)

        epoch_loss /= len(mini_batches)
        losses.append(epoch_loss)
        ### END CODE HERE ###

        # Print loss
        if print_loss and i % print_freq == 0:
            print(f"Loss after iteration {i}: {epoch_loss}")

        # Store history
        if i % 100 == 0:
            history.append((X_train, model.forward(X_train)))

        # Decrease learning rate
        if i % decrease_freq == 0 and i > 0:
            learning_rate *= decrease_proportion

    return model, losses, history
def predict(x, y_true, model):
    n = x.shape[0]

    # Forward propagation
    y_pred = model.forward(x)

    if y_pred.shape[-1] == 1:
        y_pred = np.array([[1 - y[0], y[0]] for y in y_pred])
        if y_true is not None:
            y_true = np.array([[1,0] if y == 0 else [0,1] for y in y_true.reshape(-1)])

    # make y_pred/y_true become one-hot prediction result
    if y_true is not None:
        y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    if y_true is not None:
        # compute accuracy
        correct = 0
        for yt, yp in zip(y_true, y_pred):
            if yt == yp:
                correct += 1
        print(f"Accuracy: {correct/n * 100:.2f}%")

        f1_scores = f1_score(y_true, y_pred, average=None)
        print(f'f1 score for each class: {f1_scores}')
        print(f'f1_macro score: {np.mean(np.array(f1_scores)):.2f}')

    return y_pred

def save_prediction_data(predicted_y):
    # Create DataFrame with ID, x, and y columns
    df = pd.DataFrame({
        'ID': range(len(predicted_y)),  # Add ID column starting from 0
        'y': predicted_y
    })

    # Ensure ID is the first column
    df = df[['ID', 'y']]

    # Save to CSV file
    df.to_csv('Lab4_basic_regression.csv', index=False)
    print("Prediction data saved as 'Lab4_basic_regression.csv'")

def animate_training(history, X_train, Y_train):
    fig, ax = plt.subplots()
    ax.set_xlim(0, 11)
    ax.set_ylim(-5, 5)
    line, = ax.plot([], [], 'b-', lw=1, label='Predicted')

    ground_truth_x = X_train.flatten()
    ground_truth_y = Y_train.flatten()
    ax.plot(ground_truth_x, ground_truth_y, 'r-', lw=1, label='Ground Truth')

    # show current epoch on the animation / 100 epoch
    epoch_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')

    def init():
        line.set_data([], [])
        epoch_text.set_text('')
        return line, epoch_text

    def update(frame):
        epoch = (frame + 1) * 100
        _, predicted_y = history[frame]
        predicted_x = X_train.flatten()
        line.set_data(predicted_x, predicted_y.flatten())

        epoch_text.set_text(f'Epoch: {epoch}')

        return line, epoch_text

    ani = FuncAnimation(fig, update, frames=len(history), init_func=init, blit=True, interval=50)

    # save as gif
    ani.save('Lab4_basic_regression.gif', writer='pillow')
    plt.close(fig)
    print(f"Animation saved as 'Lab4_basic_regression.gif'")


def save_final_result(model, X_train, Y_train):
    AL = model.forward(X_train)

    predicted_x = X_train.flatten()
    predicted_y = AL.flatten()

    plt.plot(predicted_x, predicted_y, 'b-', label="Predicted", lw=1)

    ground_truth_x = X_train.flatten()
    ground_truth_y = Y_train.flatten()

    save_prediction_data(predicted_y)

    plt.plot(ground_truth_x, ground_truth_y, 'r-', label='Ground Truth', lw=1)

    plt.legend()

    plt.ylim(-5, 5)
    plt.xlim(0, 11)
    plt.savefig("Lab4_basic_regression.jpg")
    plt.show()
    print("Prediction saved as 'Lab4_basic_regression.jpg'")


In [24]:
loss_function = 'cross_entropy'
layers_dims = [x_train.shape[1], 64, 32, y_train.shape[1]]
activation_fn = ['relu', 'relu', 'sigmoid']
learning_rate = 0.01
num_iterations = 10
print_loss = True
print_freq = 1
decrease_freq = 10
decrease_proportion = 0.8
# You might need to use mini_batch to reduce training time in this part
batch_size = 32

model = Model(layers_dims, activation_fn, loss_function)
model, losses, history = train_model(model, x_train, y_train, learning_rate, num_iterations, batch_size, print_loss, print_freq, decrease_freq, decrease_proportion)
print(model.parameters['W'])
print(model.parameters['b'])

KeyboardInterrupt: 

In [1]:
pred_train = predict(x_train, y_train, model)
pred_val = predict(x_val, y_val, model)

TESTING

In [22]:
weight = np.load('NN_model.npy', allow_pickle=True)

print(weight)

loss_function = 'cross_entropy'
layers_dims = [32400, 64, 32, 1]
activation_fn = ['relu', 'relu', 'sigmoid']
model = Model(layers_dims, activation_fn, loss_function)

for i in range(len(model.linear)):
    model.linear[i].parameters = weight[i]


[{'W': array([[0.0026480805055994834, 0.010699748399948347,
         0.006815858845622935, ..., 0.010831220096447472,
         0.01080442140895309, -0.011211408346019842],
        [0.007217906185357375, -0.006387703849165259, 0.01281832903684792,
         ..., 0.0006041329708801531, 0.00586541294902867,
         0.007848943932718608],
        [0.004169152713068626, 0.011757057299996343,
         -0.0021151273030579096, ..., -0.012460406095191497,
         0.012984484773547115, 0.001293673278674988],
        ...,
        [0.004238618057481547, 0.012896300776843603, 0.011302071082975266,
         ..., 0.005388704447516456, 0.007508521143838472,
         0.0025147200707836305],
        [-0.006500030817814041, -0.012216619844361275,
         0.007148974405639742, ..., 0.010496406793681573,
         -0.012572921372953561, -0.00040203057085737184],
        [-0.010634687146003196, -0.0013813576927756191,
         0.0002024907982059986, ..., 0.0048791386396175975,
         0.000986409699339043

In [ ]:
print("begin")
preprocess_folder_test = "C:/Users/user/Desktop/photo"

preprocess_test = np.array(read_images_from_folder(preprocess_folder_test), dtype="object")
print(preprocess_test.shape)

test_labels = np.ones((preprocess_test.shape[0], 1))
test_labels[0] = 0
test_labels[2] = 0
test_labels[3] = 0
test_labels[5] = 0
test_labels[9] = 0
test_labels[10] = 0
test_labels[11] = 0
# test_labels[0] = -1
# test_labels[2] = -1
# test_labels[3] = -1
# test_labels[5] = -1
# test_labels[9] = -1
# test_labels[10] = -1
# test_labels[11] = -1

print(test_labels)

pred_test = predict(preprocess_test, test_labels, model)
print(pred_test)

begin
reading from C:/Users/user/Desktop/photo...
finish reading from C:/Users/user/Desktop/photo
(12, 32400)
[[-2.]
 [ 1.]
 [-2.]
 [-2.]
 [ 1.]
 [-2.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-2.]
 [-2.]
 [-2.]]
Accuracy: 100.00%
f1 score for each class: [1.]
f1_macro score: 1.00
[1 1 1 1 1 1 1 1 1 1 1 1]
